In [4]:
# import pandas 
import pandas as pd

In [5]:
# import data 
MosAIC = pd.read_csv("September_MosAIC_LSTM_UW_Metadata_LB2.csv")
MosAIC.head()

dRepClusters = pd.read_csv("Cdb.csv")
dRepClusters.head()

VF_Genes = pd.read_csv("210423_MosAIC_VF_Search.tsv", delimiter="\t")
VF_Genes.head()

,#FILE,SEQUENCE,START,END,STRAND,GENE,COVERAGE,COVERAGE_MAP,GAPS,%COVERAGE,%IDENTITY,DATABASE,ACCESSION,PRODUCT,RESISTANCE
0,AA010_contigs.fa,contig00001,113766,114708,-,VFG037748(gb|YP_001085628),1-943/984,===============,0/0,95.83,82.72,VFDB_full,NaN,VFG037748(gb|YP_001085628) (lpxM) lipid A bios...,NaN
1,AA010_contigs.fa,contig00001,380608,382042,-,VFG037735(gb|WP_000633124),6-1440/1449,===============,0/0,99.03,84.18,VFDB_full,NaN,VFG037735(gb|WP_000633124) (adeH) outer membra...,NaN
2,AA010_contigs.fa,contig00001,382060,385239,-,VFG037724(gb|WP_001027061),1-3180/3180,===============,0/0,100.00,91.26,VFDB_full,NaN,VFG037724(gb|WP_001027061) (adeG) cation/multi...,NaN
3,AA010_contigs.fa,contig00001,385246,386466,-,VFG037705(gb|WP_000010636),1-1221/1221,===============,0/0,100.00,87.31,VFDB_full,NaN,VFG037705(gb|WP_000010636) (adeF) membrane-fus...,NaN
4,AA010_contigs.fa,contig00001,399065,400822,+,VFG050781(gb|WP_001020905.1),1-1758/1758,===============,0/0,100.00,85.67,VFDB_full,NaN,VFG050781(gb|WP_001020905.1) (gspE2) putative ...,NaN


In [6]:
# filter MosAIC df with genomes that have passed QC 
MosAIC_pass = MosAIC[MosAIC['qc_pass'] == 'Pass']
print(MosAIC_pass)

    internal_id seq_run seq_sumission_date data_return_date  \
0       USHLN06    UW-1          29-Sep-21        01-Nov-21   
1       USHLN08    UW-1          29-Sep-21        01-Nov-21   
2      USHLN005    UW-2          07-Jul-22        19-Aug-22   
3       USHLN16    UW-1          29-Sep-21        01-Nov-21   
4      USHLN010    UW-3          23-Aug-22        09-Sep-22   
..          ...     ...                ...              ...   
413     USHLN33    UW-1          29-Sep-21        01-Nov-21   
415     USHLN34    UW-1          29-Sep-21        01-Nov-21   
416     USHLN35    UW-1          29-Sep-21        01-Nov-21   
417    USHLN028    UW-2          07-Jul-22        19-Aug-22   
418    USHLN029    UW-2          07-Jul-22        19-Aug-22   

    internal_date_isolate_received     source_lab isolation_date  \
0                        06-May-21  Brackney_Doug      08-Jun-18   
1                        06-May-21  Brackney_Doug      13-Jun-18   
2                        06-May-21  Bra

In [7]:
# function to remove _contigs.fa extension 

def remove_file_extension(df, column_name):
    '''
    df = you dataframe
    column_name = target column name in ' '
    '''
    for column in [column_name]: 
        df[column_name] = df[column_name].str.replace('_contigs.fa', '')

    return(df)

VF_Genes_Clean = remove_file_extension(VF_Genes, '#FILE')
dRep_Clusters_Clean = remove_file_extension(dRepClusters, 'genome')

print(VF_Genes_Clean.head())
print("\n", dRep_Clusters_Clean.head())



   #FILE     SEQUENCE   START     END STRAND                          GENE  \
0  AA010  contig00001  113766  114708      -    VFG037748(gb|YP_001085628)   
1  AA010  contig00001  380608  382042      -    VFG037735(gb|WP_000633124)   
2  AA010  contig00001  382060  385239      -    VFG037724(gb|WP_001027061)   
3  AA010  contig00001  385246  386466      -    VFG037705(gb|WP_000010636)   
4  AA010  contig00001  399065  400822      +  VFG050781(gb|WP_001020905.1)   

      COVERAGE     COVERAGE_MAP GAPS  %COVERAGE  %IDENTITY   DATABASE  \
0    1-943/984  ===============  0/0      95.83      82.72  VFDB_full   
1  6-1440/1449  ===============  0/0      99.03      84.18  VFDB_full   
2  1-3180/3180  ===============  0/0     100.00      91.26  VFDB_full   
3  1-1221/1221  ===============  0/0     100.00      87.31  VFDB_full   
4  1-1758/1758  ===============  0/0     100.00      85.67  VFDB_full   

   ACCESSION                                            PRODUCT  RESISTANCE  
0        NaN  

In [8]:
# Evaluate file deliniators
'''
compare the ID names
'''
set1 = set(MosAIC_pass['internal_id'])
set2 = set(dRepClusters['genome'])
set3 = set(VF_Genes_Clean['#FILE'])
'''
file name has _contigs.fa
'''

print(set2.difference(set1))
print(set3.difference(set1))




{'MMO-59', 'MMO-93'}
set()


In [9]:
# join data 
MosAIC_and_VirulenceFactors = pd.merge(MosAIC_pass, VF_Genes_Clean, left_on='internal_id', right_on='#FILE', how = 'left')
print(MosAIC_and_VirulenceFactors.columns)

# select specific columns
MosAIC_VF_select = MosAIC_and_VirulenceFactors[['internal_id', 'source_lab', 'sample_source', 'lab_field_derived', 
                             'mosquito_tissue', 'mosquito_species','mosquito_female_feeding_status',
                             'mosquito_sex', 'gtdb_genus', 'gtdb_species', 'GENE', 'ACCESSION', 'PRODUCT', 'RESISTANCE']]

print(MosAIC_VF_select)



Index(['internal_id', 'seq_run', 'seq_sumission_date', 'data_return_date',
       'internal_date_isolate_received', 'source_lab', 'isolation_date',
       'sample_location', 'sample_location_lat', 'sample_location_lon',
       'sample_source', 'lab_field_derived', 'host_species_non_mosquito',
       'life_stage_non_mosquito', 'sex_non_mosquito', 'tissue_non_mosquito',
       'mosquito_species', 'mosquito_strain_lab', 'mosquito_life_stage',
       'mosquito_sex', 'mosquito_female_feeding_status', 'mosquito_tissue',
       'confirmed_growth_media', 'confirmed_growth_temp', 'qc_pass',
       'completeness', 'contamination', 'genome_size_bp', 'number_contigs',
       'mean_contig_length_bp', 'longest_contig_bp', 'gc',
       'number_predicted_genes', 'n50', 'total_reads', 'avg_coverage',
       'gtdb_domain', 'gtdb_parent', 'gtbd_class', 'gtdb_order', 'gtdb_family',
       'gtdb_genus', 'gtdb_species', 'fastani_reference', 'fastani_ani',
       'fastani_af', 'SR_accession_no', 'ER_accessio

In [12]:
# group by 
''''
Summarise the number of MosAIC virulence factors across samples, species, tissues, source
'''
MosAIC_VF_Summary_Tisssue = MosAIC_VF_select[['internal_id', 'gtdb_species',
                                               'PRODUCT', 'mosquito_tissue',
                                                'sample_source']].groupby(['gtdb_species', 'internal_id', 'mosquito_tissue','sample_source']).count()


MosAIC_VF_Summary_Tisssue.to_csv('140923_MosAIC_VF_Summary_Tissue.csv')
MosAIC_VF_Summary_Tisssue



PRODUCT
gtdb_species                   internal_id mosquito_tissue sample_source                     
Achromobacter ruhlandii        MMO-104     whole_body      mosquito                        31
                               MMO-123     whole_body      mosquito                        31
                               MMO-124     whole_body      mosquito                        31
                               MMO-144     whole_body      mosquito                        31
                               MMO-82      whole_body      mosquito                        31
...                                                                                       ...
Stenotrophomonas maltophilia   MMO-2       whole_body      mosquito                         5
                               MMO-6       whole_body      mosquito                         5
Stenotrophomonas maltophilia_A USHLN265    midgut          mosquito                         4
                               USMM131     midgut          mosquito                         4
UBA7405 sp000755535            AS183       whole_body      non_mosquito_Diptera_host       43

[283 rows x 1 columns]